Detect when the picture on screen disappears, Mark down the time and select out the +- 5 slices. 

In [12]:
import cv2
import numpy as np
import os
import glob
from datetime import datetime, timedelta

# Define the folder path containing the videos
folder_path = r'C:\Users\XuHaoxin\Videos\Trial'
template_path = r'D:\Codes\Exp_marmo_co2\img\yellow_fly.png'

# Load the template image
template = cv2.imread(template_path, 0)
if template is None:
    raise FileNotFoundError(f"Template image not found at path: {template_path}")
w, h = template.shape[::-1]

# Upload the template to the GPU
template_gpu = cv2.cuda_GpuMat()
template_gpu.upload(template)

# Get a list of all video files in the folder
video_files = glob.glob(os.path.join(folder_path, '*.mov'))  # Adjust the extension to .mov

for video_path in video_files:
    # Extract the start time from the filename
    filename = os.path.basename(video_path)
    start_time_str = filename.split('_')[1].split('.')[0]
    start_time = datetime.strptime(start_time_str, '%H%M%S')

    # Load the video
    cap = cv2.VideoCapture(video_path)

    # Initialize variables
    frame_count = 0
    frames_with_template = []
    disappearance_frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Convert the frame to grayscale and upload to GPU
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray_frame_gpu = cv2.cuda_GpuMat()
        gray_frame_gpu.upload(gray_frame)

        # Apply template matching on GPU
        res_gpu = cv2.cuda.matchTemplate(gray_frame_gpu, template_gpu, cv2.TM_CCOEFF_NORMED)
        res = res_gpu.download()

        threshold = 0.8
        loc = np.where(res >= threshold)

        # Check if template is detected in the frame
        if len(loc[0]) > 0:
            frames_with_template.append(frame_count)
        else:
            if frames_with_template and (not disappearance_frames or frame_count > disappearance_frames[-1] + 30):
                disappearance_frames.append(frame_count)

    cap.release()

    if disappearance_frames:
        print(f"Picture disappeared at frames: {disappearance_frames} in video {filename}")
    else:
        print(f"Picture did not disappear in the video {filename}.")

    # Save the slices of frames for each disappearance event
    for idx, disappearance_frame in enumerate(disappearance_frames):
        start_frame = max(0, disappearance_frame - 30)  # 30 frames before disappearance
        end_frame = disappearance_frame + 30  # 30 frames after disappearance

        cap = cv2.VideoCapture(video_path)
        frame_count = 0
        output_slice_path = os.path.join(folder_path, f'output_slice_{idx}_{filename}')
        out = cv2.VideoWriter(output_slice_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (int(cap.get(3)), int(cap.get(4))))

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret or frame_count > end_frame:
                break

            frame_count += 1
            if start_frame <= frame_count <= end_frame:
                out.write(frame)

        cap.release()
        out.release()

        # Calculate the disappearance time
        fps = cap.get(cv2.CAP_PROP_FPS)
        disappearance_time_seconds = disappearance_frame / fps
        disappearance_time = start_time + timedelta(seconds=disappearance_time_seconds)
        print(f"Disappearance time for event {idx} in video {filename}: {disappearance_time.strftime('%Y%m%d_%H%M%S')}")

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\core\include\opencv2/core/private.cuda.hpp:106: error: (-216:No CUDA support) The library is compiled without CUDA support in function 'throw_no_cuda'


In [15]:
import torch

# Check if GPU is available
print("Is CUDA available:", torch.cuda.is_available())

Is CUDA available: True
